# Look for B0 candidates

Import the required modules

In [1]:
import sys
import basf2 as b2
# Convenience modules
import modularAnalysis as ma
# Convenience functions to filter standard particles
import stdV0s
# To print statistics
from basf2 import statistics

Select the input file number from the examples to analyze and if we want to run the version with cuts (without background) or without cuts (with background)

In [2]:
filenumber = 1
include_cuts = True

## Create a Path and add a processig pipeline to it
This will be define the container of the series of steps we are going to perform to do our analysis. All steps will be added as "modules" to this path

In [3]:
main = b2.Path()

Now, we can start adding modules to the processing pipeline (i.e. to the Path)

### Load input data from mdst/udst file
We are going to add a module to the "main" path to indicate the input file containing all the events

In [4]:
ma.inputMdstList(
    environmentType="default",
    filelist=[b2.find_file(f"starterkit/2021/1111540100_eph3_BGx0_"
              f"{filenumber}.root", "examples")],
    path=main,
)

Now, we are going to add the steps that actually filter the data. Each filter is looking for a specific particle and each candidate (i.e. the result of every filter) is going to be stored in the ParticleList. The goal is to create filters and recontruct everything we need so, in the end, we can find B0 candidates.

### Load electrons, positrons.
First, we are going to look for electrons and positrons. That is, we are going "fill the particle list" with electron/positron candidates based on our filtering criteria (i.e. based on "cuts")

In [5]:
ma.fillParticleList(
    decayString="e+:uncorrected",
    cut="electronID>0.1 and dr<0.5 and abs(dz)<2 and thetaInECLAcceptance",
    path=main
)

### Load k_shorts
Then, we going to look for K_short candidates. The appropiate cuts and filters are already programmed and store in a convenience function called "stdKshorts" (which is in the module of convenience filters for Standard Particles: stdV0s).

In its inner workings, this function applies the recommended cuts (it looks for the K_S0 by looking for its product decays: K_S0 -> pi+ pi-. It looks for them using two different methods [V0 and ParticleCombiner] and then merges the results).

K_S0 candidates in a variable called K_S0:merged in the ParticleList

In [6]:
stdV0s.stdKshorts(path=main)

### Reconstruct J/psi
Now, we are looking for J/psi candidates. There is no convenience function, but it is fairly easy to look for the J/psi by its products e+ e-. The syntax for the decayString is:

````
parent:label -> children
````
 
where "label" is an optional label to add to the variable in the ParticleList If no label is provided, then the variable is just called "parent".

The cut we are using is setting a mass minus nominal mass lower than 0.11

In [7]:
ma.reconstructDecay(
    decayString="J/psi -> e+:uncorrected e-:uncorrected",
    cut="dM < 0.11",
    path=main
)

### Reconstruct B0

Up to this point, we have candidates for the J/psi and the K_S0 (stored in the variable K_S0:merged), we can use those them to create a new candidate for the B0 particle.

We are adding a cut to remove candidates that:

1. Have a beam-constrained energy greater than 5.2
2. $ |\Delta E| < 0.15$

In [8]:
ma.reconstructDecay(
    "B0 -> J/psi K_S0:merged",
    cut="abs(deltaE) < 0.15 and Mbc > 5.2" if include_cuts else "",
    path=main,
)

### Checking if it is a signal candidate or background (only available for MonteCarlo data)

Add MC matching for all particles of the decay chain and add the information whether the reconstructed B meson is a signal candidate to the ntuple. 

`matchMCTruth` stores the result in a variable called `isSignal`

In [9]:
ma.matchMCTruth(
    "B0",
    path=main
)

### Save all the kinematic variables of this events

To gain access to the kinematic variables, we need to add the module `buildEventKinematics` and set the target particle we want its kinematics to be reconstructed.

In [10]:
ma.buildEventKinematics(
    ["B0"],
    path=main
)

### Store our results to perform offline analysis. 

A good variable to start with is the beam-constrained mass Mbc, which will be stored in a tuple. This variable:

1. is only of interest for B0 candidates, so it will be one entry per candidate which is defined by the decayString

2. has to peak at the mass of the B0 (which will mean filters are indeed working and we have good candidates)

3. will be stored in a file called "Bd2JpsiKS.root" in a tree called "tree"

As the data comes from a MC simulation, we know if the candidates are correct or not with the `isSignal` variable, so we are going to store it as well.

We will store other variables so, for the process:

$$ B_0 \rightarrow (J/\Psi \rightarrow e^+ e^-) (K_s^0 \rightarrow \pi^+ \pi^-) $$

We will end up storing:

* For all particles : standard variables (reconstructed kinematics in the lab frame `vc.kinematics`, truth kinematics in the lab frame `vc.mc_kinematics`) and the kinematics in the CMS `useCMSFrame(kinematics)`.
* For the $B_0$: Mbc and deltaE `vc.deltae_mbc`
* For the resonances $J/\Psi$ and $K_S^0$: the invariant mass `vc.inv_mass`
* For the final state charged particles (electron, positron and pions): PID and track data

In [11]:
import variables.collections as vc

""" define the variables that will be applied to all particles"""
# store the kinematic variables
variables = vc.kinematics + vc.mc_kinematics 
# store the isSignal variable
variables += vc.mc_truth
# add specific variables to B0
b_vars = variables + vc.deltae_mbc

import variables.utils as vu
# add PID and track variables for all charged particles in the final
# states
b_vars += vu.create_aliases_for_selected(
    list_of_variables = variables + vc.pid + vc.track + vc.track_hits,
    decay_string="B0 -> [J/psi -> ^e+ ^e-] [K_S0 -> ^pi+ ^pi-]",
    prefix=["ep", "em", "pip", "pim"],
)

# add the invariant mass of the intermediate resonances to the ntuple
b_vars += vu.create_aliases_for_selected(
    list_of_variables = variables + vc.inv_mass,
    decay_string="B0 -> ^J/psi ^K_S0",
)

# Add the kinematics in the CMS frame for all particles
cmskinematics = vu.create_aliases(
    list_of_variables=vc.kinematics,
    wrapper="useCMSFrame({variable})",
    prefix="cms"
)
b_vars += vu.create_aliases_for_selected(
    list_of_variables = cmskinematics,
    decay_string = "^B0 -> [^J/psi -> ^e+ ^e-] [^K_S0 -> ^pi+ ^pi-]"
)

# store results in a root file

ma.variablesToNtuple(
    decayString="B0", variables=b_vars,
    filename=f"Bd2JpsiKS_{filenumber}_{'with_cut' if include_cuts else 'without_cut'}.root",
    treename="tree",
    path=main
)

Welcome to JupyROOT 6.20/04


In [12]:
from variables import variables as vm
vm.printAliases()

[INFO] =====================================
[INFO] The following aliases are registered:
[INFO] Jpsi_E           --> daughter(0,E)
[INFO] Jpsi_ErrM        --> daughter(0,ErrM)
[INFO] Jpsi_InvM        --> daughter(0,InvM)
[INFO] Jpsi_M           --> daughter(0,M)
[INFO] Jpsi_SigM        --> daughter(0,SigM)
[INFO] Jpsi_cms_E       --> daughter(0,cms_E)
[INFO] Jpsi_cms_p       --> daughter(0,cms_p)
[INFO] Jpsi_cms_pt      --> daughter(0,cms_pt)
[INFO] Jpsi_cms_px      --> daughter(0,cms_px)
[INFO] Jpsi_cms_py      --> daughter(0,cms_py)
[INFO] Jpsi_cms_pz      --> daughter(0,cms_pz)
[INFO] Jpsi_e_0_cms_E   --> daughter(0,daughter(0,cms_E))
[INFO] Jpsi_e_0_cms_p   --> daughter(0,daughter(0,cms_p))
[INFO] Jpsi_e_0_cms_pt  --> daughter(0,daughter(0,cms_pt))
[INFO] Jpsi_e_0_cms_px  --> daughter(0,daughter(0,cms_px))
[INFO] Jpsi_e_0_cms_py  --> daughter(0,daughter(0,cms_py))
[INFO] Jpsi_e_0_cms_pz  --> daughter(0,daughter(0,cms_pz))
[INFO] Jpsi_e_1_cms_E   --> daughter(0,daughter(1,cms_E))
[

[INFO] ep_px            --> daughter(0,daughter(0,px))
[INFO] ep_py            --> daughter(0,daughter(0,py))
[INFO] ep_pz            --> daughter(0,daughter(0,pz))
[INFO] ep_z0            --> daughter(0,daughter(0,z0))
[INFO] pim_E            --> daughter(1,daughter(1,E))
[INFO] pim_d0           --> daughter(1,daughter(1,d0))
[INFO] pim_deuteronID   --> daughter(1,daughter(1,deuteronID))
[INFO] pim_dr           --> daughter(1,daughter(1,dr))
[INFO] pim_dx           --> daughter(1,daughter(1,dx))
[INFO] pim_dy           --> daughter(1,daughter(1,dy))
[INFO] pim_dz           --> daughter(1,daughter(1,dz))
[INFO] pim_electronID   --> daughter(1,daughter(1,electronID))
[INFO] pim_isSignal     --> daughter(1,daughter(1,isSignal))
[INFO] pim_kaonID       --> daughter(1,daughter(1,kaonID))
[INFO] pim_mcE          --> daughter(1,daughter(1,mcE))
[INFO] pim_mcErrors     --> daughter(1,daughter(1,mcErrors))
[INFO] pim_mcP          --> daughter(1,daughter(1,mcP))
[INFO] pim_mcPDG        --> daug

## Perform processing

In [13]:
# Start the event loop (actually start processing things)
b2.process(main)

In [14]:
# print results stored in global statistics
print(statistics)

[ERROR] ProcessStatistics data object is not available, you either disabled statistics with --no-stats or didn't run process(path) yet.

